<h1 style="text-align: center;">PROJECT NEO AISSYR</h1>

<img 
    style="display: block; 
           margin-left: auto;
           margin-right: auto;
           width: 40%;"
    src="/Users/charley.dlebarbier/Documents/Dev/VSCode/Projects/projet_NeoAIssyr/extra/cover.jpg" 
    alt="æventyr illustration">
</img>

<h3 style="margin-left: 80px; font-style: bold;">ABSTRACT : FR - EN</h3>
<p style="text-align: justify; margin-left: 80px; margin-right: 80px;">
Neo AISSYR est une <b>web application</b> permettant aux chercheurs, spécialistes ou curieux de l'empire Assyrien, de transcrire les tablettes en cunéiforme de la période dite Néo-Assyrienne s'étendant entre 900 et 600 ans avant notre ère.

<p style="text-align: center;">-*-</p>
<p style="text-align: justify; margin-left: 80px; margin-right: 80px;">
Neo AISSYR is a <b>web application</b> enabling researchers, specialists or those curious about the Assyrian Empire, to transcribe cuneiform tablets from the so-called Neo-Assyrian period stretching from 900 to 600 BC.

</br>

<p style="text-align: center;">-*-</p>
<h4 style="margin-left: 80px; font-style: bold;">Source dataset</h4>
<p style="text-align: justify; margin-left: 80px; margin-right: 80px;">
<a href="https://github.com/CompVis/cuneiform-sign-detection-dataset">Github Dataset</a></p>
<p style="text-align: justify; margin-left: 80px; margin-right: 80px;">
Introduit dans : <em>Dencker, T., Klinkisch, P., Maul, S. M., and Ommer, B. (2020): Deep Learning of Cuneiform Sign Detection with Weak Supervision using Transliteration Alignment, PLOS ONE, 15:12, pp. 1–21</em> <a href="https://doi.org/10.1371/journal.pone.0243039">article link</a></p>

---


<h4 style="text-align: center;">NOTEBOOK PARTIE 2 - PART 2</h4>
<p style="text-align: center; font-style: italic">~~ OBTENIR LES IMAGES - DOWNLOAD IMAGES ~~</p>
</br>

<p style="text-align: justify; margin-left: 80px; margin-right: 80px;">
<p style="text-align: justify; margin-left: 80px; margin-right: 80px;">
Ce projet s'appuie sur le dataset présenté dans l'article <em>Dencker, T., Klinkisch, P., Maul, S. M., and Ommer, B. (2020): Deep Learning of Cuneiform Sign Detection with Weak Supervision using Transliteration Alignment, PLOS ONE, 15:12, pp. 1–21</em> est disponible sur <a href="https://github.com/CompVis/cuneiform-sign-detection-dataset">Github</a>. L'ensemble des données repose sur plus de 1800 images de tablettes d'argile de l'époque néo-assyrienne de l'écriture cunéiforme (900 - 600 ans AV. N. È.). Le dataset, fournit les Bounding Box de 8109 signes labellisés en 189 classes, réparties sur 81 tablettes.</p>
</p>

<p style="text-align: justify; margin-left: 80px; margin-right: 80px;">
Dans un <a href="notebook_part1_-_EDA_DATASET.ipynb">premier notebook</a>, l'analyse des fichiers CSV du dataset a été effectuée. Cependant, étant donné l'absence des images des tablettes dans les dossiers, il est nécessaire de les télécharger depuis une source externe. Une opération de web scraping sera effectuée sur le site web de la Cuneiform Digital Library Initiative (CDLI), un groupe international d'assyriologues, de conservateurs de musées et d'historiens qui met à disposition les artefacts en cunéiforme. Cette démarche vise à récupérer les tablettes indispensables à l'entraînement du modèle d'IA. Tout comme pour le premier notebook, une analyse approfondie des images sera effectuée en les mettant en parallèle des fichiers CSV.
</p>

</br>

---
</br>

In [1]:
import os
import pandas as pd
import urllib.request, urllib.error

from glob import glob
from tqdm import tqdm

## Récupérer les noms des tablets des datasets 

Téléchargement des images depuis le site CDLI (Cuneiform Digital Library Initiative). Les images provenant du Vorderasiatisches Museum Berlin (libellé VAT) proviennent d'une archive zip disponible depuis le lien source.

Pour le moment on ne se base pas sur la construction train et test déjà présente dans le dataset, nous allons au contraire télécharger l'ensemble des tablets puis par la suite décomposer celles-ci sur notre propre train/test split.

In [9]:
FOLDER_PATH = '../data/annotations/'

## - VAT Tablet download from zip not available in CDLI database
vat_tablets = ['VAT08803Rs', 'VAT08803Vs', 'VAT10601', 'VAT10657', 
               'VAT11100Rs', 'VAT13604Rs', 'VAT13604Vs']

tablets_list = []

In [10]:
def get_tablet_references(folder_path: str) -> None:
    """
    Retrieves unique tablet references from a collection of CSV files in the specified folder.

    Parameters
    ----
    - folder_path (str, required): The path to the folder containing CSV files.

    Returns
    ----
    None

    The function reads each CSV file in the specified folder, extracts unique tablet references from the 'tablet_CDLI' column,
    and appends them to the global 'tablets_list'.

    The tablet references are filtered to include only those not already present
    in 'tablets_list' and other control list (vat_tablets).
    """
    files_path = glob(os.path.join(folder_path, 'bbox*.csv'))

    for file in files_path:
        df = pd.read_csv(file)

        tablets_from_file = df['tablet_CDLI'].unique()
        tablets_list.extend(set(tablets_from_file) - set(tablets_list) - set(vat_tablets))

In [11]:
get_tablet_references(FOLDER_PATH)

Une fois l'ensemble des tablettes récupérées depuis les csv de références, nous pouvons télécharger les images. 

Cependant, deux cas de figures peuvent se présenter :
- la référence de la tablette est introuvable (changement de référence ou suppression de la base)
- erreur 500

Nous prévoyons une liste de récupération des tablettes qui n'ont pu être téléchargées, et ce, afin de contrôler manuellement l'existence de ces références sous un autre nom.

In [17]:
IMAGE_FOLDER = '../data/images/CDLI/'
CDLI_URL = 'https://cdli.ucla.edu/dl/photo/{}.jpg'

failed_tablets = []

In [15]:
def download_img(url: str, file_path: str, file_name: str, format: str='jpg'):
    """Download Image from Internet via URL (can be dynamic) using urllib
    Params
    ----
    - url: String (required)
    - file_path: String (required), where the file will be saved
    - file_name: String (required), the name for the file
    - format: String (default: jpg)
    """
    saved_path = f'{file_path}{file_name}.{format}'

    try:
        urllib.request.urlretrieve(url, saved_path)

    except urllib.error.URLError as e:
        print(f"Error downloading {file_name}: {e}")
        failed_tablets.append(file_name)

In [18]:
for tablet in tqdm(tablets_list):
    img_url = CDLI_URL.format(tablet)
    download_img(img_url, IMAGE_FOLDER, tablet)

  3%|▎         | 2/74 [00:02<01:06,  1.08it/s]

Error downloading K01057: HTTP Error 500: Internal Server Error


 47%|████▋     | 35/74 [00:31<00:24,  1.58it/s]

Error downloading P336663b: HTTP Error 500: Internal Server Error


 89%|████████▉ | 66/74 [01:00<00:05,  1.53it/s]

...

 92%|█████████▏| 68/74 [01:01<00:03,  1.97it/s]

Error downloading BM099070: HTTP Error 500: Internal Server Error


 97%|█████████▋| 72/74 [01:05<00:01,  1.19it/s]

Error downloading K09237Vs: HTTP Error 500: Internal Server Error


100%|██████████| 74/74 [01:08<00:00,  1.08it/s]


In [19]:
failed_tablets

['K01057', 'P336663b', 'K08396', 'ND02486', 'BM099070', 'K09237Vs']

Les tablettes suivantes n'ont pas été trouvées sur le site via le script :
- K01057, 
- P336663b, 
- K08396, 
- ND02486, 
- BM099070, 
- K09237Vs

Il se peut que la référence de l'url est un autre nom.
Pour cela, nous vérifions manuellement si ces tablettes sont toujours présentes sur le site ou non.

URL : https://cdli.mpiwg-berlin.mpg.de/search?simple-value%5B%5D=&simple-field%5B%5D=


Après vérification :

- K01057 -> P334042
- P336663b -> Référence P336663 existante, mais mauvaise prise de vue 
- K08396 -> P394941
- ND02486 -> P393688
- BM099070 -> P336615
- K09237Vs -> Référence K09237 existante, mais mauvaise prise de vue

Les tablettes peuvent ainsi être téléchargées. Pour des raisons de correspondance avec les csv, le nom originel des tablettes sera préservé.

In [22]:
dict_failed_tablet = {
    'K01057': 'P334042', 
    'K08396': 'P394941',
    'ND02486': 'P393688',
    'BM099070': 'P336615'
}

for key in tqdm(dict_failed_tablet):
    img_url = CDLI_URL.format(dict_failed_tablet[key])
    download_img(img_url, IMAGE_FOLDER, key)

100%|██████████| 4/4 [00:02<00:00,  1.60it/s]
